## Pythonによる画像解析実例 #5: 出芽酵母位相差像の分節化と蛍光granuleの解析  
\#3で確立した画像解析を修正、より細かな細胞内顆粒（eg. Ade4-mNG顆粒）を検出可能にする。  
具体的にはfociを検出する場合はz-stackをMax int. projectionした像をそのまま使用していたが、projection後にRolling ball background, rolling = 10でバックグラウンドを差し引いてから解析。  
位相差像から細胞の輪郭を検出するプログラムに変更無し。

In [8]:
import numpy as np
import scipy as sp
import pandas as pd
import glob
import nd2
import os
import imageio
from tifffile import tifffile as tiff
import PySimpleGUI as sg

import matplotlib.pyplot as plt
from scipy import ndimage

import cv2
from cv2_rolling_ball import subtract_background_rolling_ball

from skimage.color import label2rgb
from skimage.color import rgb2gray
from skimage import exposure
from skimage.filters import threshold_li
from skimage.measure import regionprops_table
from skimage.measure import label
from skimage.morphology import binary_opening
from skimage.morphology import binary_dilation
from skimage.morphology import extrema
from skimage.morphology import remove_small_objects
from skimage.morphology import square
from skimage.morphology import skeletonize
from skimage import restoration
from skimage.segmentation import clear_border
from skimage.util import img_as_ubyte
from skimage.util import img_as_uint
from skimage.util import invert
# from skimage.util import io

位相差像のラベル化画像の作成までを関数化

In [2]:
def segm_phimg(img0, radius = 25, ratio = 0.1):

    img1 = img0[1,:,:]
    gblur = cv2.GaussianBlur(
        img1,    # 入力画像
        (0, 0), # カーネルのサイズを０にしてσの値からカーネルサイズを自動計算
        1       # X方向のσの値, Y方向のσを指定しないとXと同じ値になる
        )
    background = restoration.rolling_ball(gblur, radius = radius, kernel=None,
                                     nansafe=False, num_threads=None)
    filtered = gblur - background
    
    thresh = threshold_li(filtered)
    mask = np.where(filtered > thresh, 1, 0) # numpyによる2値化処理

    arr = mask > 0
    cleaned = remove_small_objects(arr, min_size=256)
    cleaned = img_as_ubyte(cleaned)

    erosion = ndimage.binary_erosion(cleaned, iterations= 2).astype(cleaned.dtype)

    sk = skeletonize(erosion, method='lee')
    sk_dil = ndimage.binary_dilation(sk, iterations= 1).astype(sk.dtype)
    sk_close = ndimage.binary_closing(sk_dil, iterations=1).astype(sk_dil.dtype)

    filled = ndimage.binary_fill_holes(sk_close).astype("uint8")
    diff = filled - sk_close
    filled2= ndimage.binary_fill_holes(diff).astype("uint8")
    sure_bg = ndimage.binary_dilation(filled2, iterations= 1).astype(filled2.dtype)

    dist_transform = cv2.distanceTransform(filled2, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, ratio*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    sure_bg = np.uint8(sure_bg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    # ret, markers = cv2.connectedComponents(sure_fg)
    ret, markers, stats, centroids = cv2.connectedComponentsWithStats(sure_fg)
    markers = markers+1
    markers[unknown == 1] = 0 # boolean indexing
    
    img1_1 = exposure.rescale_intensity(img1)  # 明るさを調整
    img1_8b = img_as_ubyte(img1_1)
    img1_8b_rgb = cv2.cvtColor(img1_8b, cv2.COLOR_GRAY2RGB)
    markers = cv2.watershed(img1_8b_rgb, markers) # imgはRGBである必要がある
    
    #オーバーレイ画像の作成
    img1 = exposure.rescale_intensity(img1)  # 明るさを調整
    img1_2 = img_as_ubyte(img1)
    img_rgb_segm = cv2.cvtColor(img1_2, cv2.COLOR_GRAY2RGB)
    img_rgb_segm[markers == -1] = (255, 0, 0)
    
    # numbered = img_rgb_segm.copy()
    # for i in range(1, len(stats)):
    #     x, y, width, height, area = stats[i]  # このAreaは何を表している？
    #     cv2.putText(numbered, f"{i}", (x, y-20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, cv2.LINE_AA)

    return markers, stats, centroids, img_rgb_segm

In [3]:
# df内のmaximaのラベル画像と各領域の元画像から、overlay画像を作るための関数
def make_overlay(df):
    return label2rgb(df["label_h_maxima"], df["image_intensity"]*3, alpha=0.2, bg_label=0,
                          bg_color=None, colors=[(1, 0, 0)])

df内で個々のmaximaを解析して、結果を新しい列に格納するための関数  
fociを持たない細胞の場合は全ての値でNaNを返す

In [4]:
def detect_maxima(df):
    if df["with_foci"] == 1:
        dic = regionprops_table(df["label_h_maxima"], intensity_image = df["image_intensity"], properties=["label", "area", "intensity_max", "intensity_mean"])
        foci_df = pd.DataFrame(dic)
    else:
        dic = {"label":np.nan, "area":np.nan, "intensity_max":np.nan, "intensity_mean":np.nan}
        foci_df = pd.DataFrame(dic, index=[0])# indexを何か指定しないとエラー
    foci_df["cell_label"] = df.cell_label
    foci_df["cell_area"] = df.cell_area
    foci_df["with_foci"] = df.with_foci
    
    return foci_df

作成したラベル化画像を入力して、各領域（各細胞）の面積やfociの有無等についてまとめたデータフレームdfと個々のfociについて計測してまとめたデータフレームdf_fociを返す関数  

引数として受け取った画像のバックグラウンドをskimageのrolling-ball methodで差し引くように変更

In [9]:
def analyze_foci(markers, img0, prominence):
    img1 = img0[0,:,:]
    # background = restoration.rolling_ball(img1, radius = 10, kernel=None,
    #                                  nansafe=False, num_threads=None)
    # img2 = img1 - background
    
    # cv2のrolling-ballを使用する場合、入力画像のimgも変更される
    img1_8b = img_as_ubyte(img1) #8bit画像に変換
    radius = 10
    img2_8b, background = subtract_background_rolling_ball(img1_8b, radius, light_background=False,
                                     use_paraboloid=False, do_presmooth=False)
    img2 = img_as_uint(img2_8b)
    
    img_properties = ["label", "area","image_intensity"]
    prop_dic = regionprops_table(markers, intensity_image=img2, properties=img_properties)
    df = pd.DataFrame(prop_dic)
    df = df.rename(columns = {"label": "cell_label"})
    df = df.rename(columns = {"area": "cell_area"})
    df = df.drop([0]) # バックグラウンドのデータを削除
    df = df.reset_index(drop=True)
    df["cell_label"] = df["cell_label"] - 1 # 画像のラベル番号と合わせるため
    
    # ImageJ/FIJIマクロと同様の解析条件:
    min_th = 400  # 解析対象のcell_areaの下限
    max_th = 3000 # 解析対象のcell_areaの上限
    
    # 別法： 平均値とSDから上限と下限を決める
    # cell_area_mean = df["cell_area"].mean()
    # cell_area_sdv = df["cell_area"].std()
    # min_th = cell_area_mean/3  # 解析対象のcell_areaの下限： 平均/3
    # max_th = cell_area_mean + 3*cell_area_sdv # 解析対象のcell_areaの上限： 平均 + 3*SD
    
    # 条件に合致する行を抽出
    labels_extracted = df[(df["cell_area"] < max_th) & (min_th < df["cell_area"])]["cell_label"]
    df = df[(df["cell_area"] < max_th) & (min_th < df["cell_area"])]
    
    df["h_maxima"] = df["image_intensity"].map(lambda x: extrema.h_maxima(x, prominence, footprint=None))
    df["num_maxima"] = df["h_maxima"].map(lambda x: x.sum()) # mapを使わないとエラー
    df["with_foci"] = df["num_maxima"].map(lambda x: 0 if x == 0 else 1)
    # さらに極大点のdilation、ラベリングまでdataframe内で行う
    # df["h_maxima_dil"]= df["h_maxima"].map(lambda x: ndimage.binary_dilation(x, iterations= 1).astype(x.dtype))
    df["h_maxima_dil"]= df["h_maxima"].map(lambda x: binary_dilation(x, footprint = square(2)).astype(x.dtype))
    df["label_h_maxima"] = df["h_maxima_dil"].map(label)
    df["overlay_h"] = df.apply(make_overlay, axis=1) # 自作関数によるオーバーレイ画像の作成
    df["foci_df"] = df.apply(detect_maxima, axis=1)#自作関数による各maximaの解析
    
    total_cells = len(df)
    cells_wfoci = len(df[df["with_foci"] == 1])
    cells_wofoci = total_cells - cells_wfoci
    pct_fcells = cells_wfoci/total_cells*100
    # データフレームに追加しやすいよう辞書形式でまとめておく
    res_dic = {"total cells": total_cells, "cells_with_foci": cells_wfoci, "pct_foci_cells":pct_fcells, "prominence":prominence}

    foci_df = pd.concat(list(df["foci_df"]))
    foci_df = foci_df.rename(columns = {"label": "foci_label"})
    foci_df = foci_df.iloc[:, [4,5,6,0,1,2,3]]
    return df, foci_df, res_dic, labels_extracted

cell_areaの条件で抽出したcellにのみ番号を振った画像を作成

In [6]:
def numbering_extracted_cells(img_rgb_segm, stats, labels_extracted):
    numbered = img_rgb_segm.copy()
    for i in range(1, len(stats)):
        if i in np.asarray(labels_extracted):
            x, y, width, height, area = stats[i]  # このAreaは何を表している？
            cv2.putText(numbered, f"{i}", (x, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, cv2.LINE_AA)
    return numbered

あるフォルダに存在するnd2ファイルを順番に読み込んで、fociを持つ細胞の割合およびfociの輝度を計測、各ファイルの結果をまとめたデータフレームを作成  

In [11]:
%%timeit -r 1 -n 1

prominence = 750
# folderS = "/Users/masak_takaine/210625_ade4GFP/nd"
folderS = "/Users/masak_takaine/221025_Ade4-mNG_HD_2ugml_digit/nd/test"
# folderD = "/Users/masak_takaine/210625_ade4GFP/nd_test_results"
folderD = "/Users/masak_takaine/221025_Ade4-mNG_HD_2ugml_digit/python"
date = "test"
file_list = glob.glob(folderS + "/*") #フォルダ内の全てのfile/folderのリストを取得して、ソートしておく
file_list.sort(reverse=True) 

# 各ファイルの解析結果を格納するリスト
fname_list = []
vsize_list =[]
res_df_list = []
foci_df_list = []
segm_img_list = []

csv_folder = os.path.join(folderD, "csv")
if not os.path.exists(csv_folder):  #dirが存在しなければ作成する
    os.mkdir(csv_folder)  

segm_img_folder = os.path.join(folderD, "segmented_images")    
if not os.path.exists(segm_img_folder):  
    os.mkdir(segm_img_folder) 

for path in file_list:
    filename = os.path.basename(path).split(".")[0]
    nd2_arr =nd2.imread(path) # 画像をndarrayとして読み込み
    with nd2.ND2File(path) as f: # with構文でファイルを開くと様々なメタデータが抽出できる
        vsize = f.voxel_size() # 
    img0 = np.max(nd2_arr, axis = 0) # z-planeを表すaxis = 0の最大値を計算
    markers, stats, centroids, img_rgb_segm = segm_phimg(img0)
    df, foci_df, res_dic, labels_extracted = analyze_foci(markers, img0, prominence)
    numbered = numbering_extracted_cells(img_rgb_segm, stats, labels_extracted)
    
    fname_list.append(filename)
    vsize_list.append(vsize)
    res_dic["filename"] = filename
    res_dic["Date"] = date
    res_df = pd.DataFrame(res_dic, index=[0])# indexを何か指定しないとエラー
    res_df = res_df.iloc[:, [5,4,0,1,2,3]] # 並べ替え
    res_df_list += [res_df] # リストは+=で簡単に連結できる
    
    foci_df["filename"] = filename
    foci_df["Date"] = date
    foci_df = foci_df.iloc[:, [7,6,0,1,2,3,4,5]]
    foci_df_list += [foci_df]
    
    segm_img_list += [numbered]

df_concat = pd.concat(res_df_list)
df_concat = df_concat.reset_index(drop=True) # indexを振り直す
stats_csv_savepath = os.path.join(csv_folder, date + "_stats.csv")
df_concat.to_csv(stats_csv_savepath)

foci_df_concat = pd.concat(foci_df_list)
foci_df_concat = foci_df_concat.reset_index(drop=True)
fdata_csv_savepath = os.path.join(csv_folder, date + "_focidata.csv")
foci_df_concat.to_csv(fdata_csv_savepath)

for filename, vsize, numbered in zip(fname_list, vsize_list, segm_img_list):
    segm_img_savepath = os.path.join(segm_img_folder, filename + "_segmented.tif")
    tiff.imwrite(segm_img_savepath, numbered, imagej = True, resolution = (1/vsize.x, 1/vsize.y), metadata={'unit': 'um'})

print("Done.")

Done.
8min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


35個のnd2ファイルの解析と測定結果の保存にかかる時間を計測:  
folderS = "/Users/masak_takaine/210625_ade4GFP/nd"  
folderD = "/Users/masak_takaine/210625_ade4GFP/nd_test_results"  
1min 1s ± 238 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)  
同じファイルをFIJIのJythonスクリプトで処理すると、約20分かかった

さらに読み込むファイルの入ったフォルダ（source folder）と保存先のフォルダ（destination folder）をGUIで指定できるようにする

In [12]:
layout = [
    [sg.Text("Date of experiments:"), sg.InputText(key="date",default_text = "test")],
   [sg.Text("Source folder:"), sg.InputText(key="folderS")],[sg.FolderBrowse(target="folderS", initial_folder= os.getcwd())],
    [sg.Text("Destination folder:"), sg.InputText(key="folderD")],[sg.FolderBrowse(target="folderD", initial_folder= os.getcwd())],
    [sg.Text("Prominence:"), sg.InputText(key="prom",default_text = "2000")],
          [sg.Submit(), sg.Cancel()],
]

window = sg.Window("Choose folders", layout)
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Cancel'):
        break
    # elif event == 'Submit':
    else:
        date = values["date"]
        folderS = values["folderS"]
        folderD = values["folderD"]
        prominence = int(values["prom"])
        file_list = glob.glob(folderS + "/*") #フォルダ内の全てのfile/folderのリストを取得して、ソートしておく
        file_list.sort(reverse=True) 

        # 各ファイルの解析結果を格納するリスト
        fname_list = []
        vsize_list =[]
        res_df_list = []
        foci_df_list = []
        segm_img_list = []

        csv_folder = os.path.join(folderD, "csv")
        if not os.path.exists(csv_folder):  #dirが存在しなければ作成する
            os.mkdir(csv_folder)  

        segm_img_folder = os.path.join(folderD, "segmented_images")    
        if not os.path.exists(segm_img_folder):  
            os.mkdir(segm_img_folder) 

        for path in file_list:
            filename = os.path.basename(path).split(".")[0]
            nd2_arr =nd2.imread(path) # 画像をndarrayとして読み込み
            with nd2.ND2File(path) as f: # with構文でファイルを開くと様々なメタデータが抽出できる
                vsize = f.voxel_size() # 
            img0 = np.max(nd2_arr, axis = 0) # Max intensity projection, z-planeを表すaxis = 0の最大値を計算
            markers, stats, centroids, img_rgb_segm = segm_phimg(img0)
            df, foci_df, res_dic, labels_extracted = analyze_foci(markers, img0, prominence)
            numbered = numbering_extracted_cells(img_rgb_segm, stats, labels_extracted)

            fname_list.append(filename)
            vsize_list.append(vsize)
            res_dic["filename"] = filename
            res_dic["Date"] = date
            res_df = pd.DataFrame(res_dic, index=[0])# indexを何か指定しないとエラー
            res_df = res_df.iloc[:, [5,4,0,1,2,3]] # 並べ替え
            res_df_list += [res_df] # リストは+=で簡単に連結できる

            foci_df["filename"] = filename
            foci_df["Date"] = date
            foci_df = foci_df.iloc[:, [7,6,0,1,2,3,4,5]]
            foci_df_list += [foci_df]

            segm_img_list += [numbered]

        df_concat = pd.concat(res_df_list)
        df_concat = df_concat.reset_index(drop=True) # indexを振り直す
        stats_csv_savepath = os.path.join(csv_folder, date + "_stats.csv")
        df_concat.to_csv(stats_csv_savepath)

        foci_df_concat = pd.concat(foci_df_list)
        foci_df_concat = foci_df_concat.reset_index(drop=True)
        fdata_csv_savepath = os.path.join(csv_folder, date + "_focidata.csv")
        foci_df_concat.to_csv(fdata_csv_savepath)

        for filename, vsize, numbered in zip(fname_list, vsize_list, segm_img_list):
            segm_img_savepath = os.path.join(segm_img_folder, filename + "_segmented.tif")
            tiff.imwrite(segm_img_savepath, numbered, imagej = True, resolution = (1/vsize.x, 1/vsize.y), metadata={'unit': 'um'})
        print("Done.")
        break
        
window.close()

Done.


さらに各チャンネルの画像のMIPをまとめた、ImageJ hyperstackファイルを作成する機能を追加

In [ ]:
layout = [
    [sg.Text("Date of experiments:"), sg.InputText(key="date",default_text = "test")],
   [sg.Text("Source folder:"), sg.InputText(key="folderS")],[sg.FolderBrowse(target="folderS", initial_folder= os.getcwd())],
    [sg.Text("Destination folder:"), sg.InputText(key="folderD")],[sg.FolderBrowse(target="folderD", initial_folder= os.getcwd())],
    [sg.Text("Prominence:"), sg.InputText(key="prom",default_text = "2000")],
    # [sg.Text("Prominence:"), sg.InputText(key="prom")],
          [sg.Submit(), sg.Cancel()],
]

window = sg.Window("Choose folders", layout)
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Cancel'):
        break
    # elif event == 'Submit':
    else:
        date = values["date"]
        folderS = values["folderS"]
        folderD = values["folderD"]
        prominence = int(values["prom"])
        file_list = glob.glob(folderS + "/*") #フォルダ内の全てのfile/folderのリストを取得して、ソートしておく
        file_list.sort(reverse=True) 

        # 各ファイルの解析結果を格納するリスト
        fname_list = []
        vsize_list =[]
        res_df_list = []
        foci_df_list = []
        segm_img_list = []
        
        ch0_list = []
        ch1_list = []

        csv_folder = os.path.join(folderD, "csv")
        if not os.path.exists(csv_folder):  #dirが存在しなければ作成する
            os.mkdir(csv_folder)  

        segm_img_folder = os.path.join(folderD, "segmented_images")    
        if not os.path.exists(segm_img_folder):  
            os.mkdir(segm_img_folder) 

        for path in file_list:
            filename = os.path.basename(path).split(".")[0]
            nd2_arr =nd2.imread(path) # 画像をndarrayとして読み込み
            with nd2.ND2File(path) as f: # with構文でファイルを開くと様々なメタデータが抽出できる
                vsize = f.voxel_size() # 
            img0 = np.max(nd2_arr, axis = 0) # Max intensity projection, z-planeを表すaxis = 0の最大値を計算
            markers, stats, centroids, img_rgb_segm = segm_phimg(img0)
            df, foci_df, res_dic, labels_extracted = analyze_foci(markers, img0, prominence)
            numbered = numbering_extracted_cells(img_rgb_segm, stats, labels_extracted)

            fname_list.append(filename)
            vsize_list.append(vsize)
            res_dic["filename"] = filename
            res_dic["Date"] = date
            res_df = pd.DataFrame(res_dic, index=[0])# indexを何か指定しないとエラー
            res_df = res_df.iloc[:, [5,4,0,1,2,3]] # 並べ替え
            res_df_list += [res_df] # リストは+=で簡単に連結できる

            foci_df["filename"] = filename
            foci_df["Date"] = date
            foci_df = foci_df.iloc[:, [7,6,0,1,2,3,4,5]]
            foci_df_list += [foci_df]

            segm_img_list += [numbered]
            
            # hyperstack作成のため、各チャンネル画像をリストに格納
            h, w = img0.shape[1], img0.shape[2]
            img0_ch0 = img0[0,:,:].reshape(1, h, w) # (frame, y, x)に次元変更
            img0_ch1 = img0[1,:,:].reshape(1, h, w)
            ch0_list.append(img0_ch0)
            ch1_list.append(img0_ch1)

        df_concat = pd.concat(res_df_list)
        df_concat = df_concat.reset_index(drop=True) # indexを振り直す
        stats_csv_savepath = os.path.join(csv_folder, date + "_stats.csv")
        df_concat.to_csv(stats_csv_savepath)

        foci_df_concat = pd.concat(foci_df_list)
        foci_df_concat = foci_df_concat.reset_index(drop=True)
        fdata_csv_savepath = os.path.join(csv_folder, date + "_focidata.csv")
        foci_df_concat.to_csv(fdata_csv_savepath)

        for filename, vsize, numbered in zip(fname_list, vsize_list, segm_img_list):
            segm_img_savepath = os.path.join(segm_img_folder, filename + "_segmented.tif")
            tiff.imwrite(segm_img_savepath, numbered, imagej = True, resolution = (1/vsize.x, 1/vsize.y), metadata={'unit': 'um'})
            
        # vstackでリスト内の画像を1つ目の軸にまとめる
        ch0_stack = np.vstack(ch0_list)
        ch1_stack = np.vstack(ch1_list)
        # 観察中に解像度が変更されることは無いので、最初のvsizeだけ使用
        vsize0 = vsize_list[0]
        reso = (1/vsize0.x, 1/vsize0.y)
        tiff.imwrite(os.path.join(folderD,'ch0_stack.tif'),ch0_stack,imagej=True,resolution= reso,
             metadata={
                # 'spacing': 3.947368,
                 'unit': 'um',
                 # 'finterval': 1/10,
                 # 'fps': 10.0,
               'axes': 'TYX',
                 'Labels': fname_list,
                     })
        tiff.imwrite(os.path.join(folderD,'ch1_stack.tif'),ch1_stack,imagej=True,resolution= reso,
             metadata={
                # 'spacing': 3.947368,
                 'unit': 'um',
                 # 'finterval': 1/10,
                 # 'fps': 10.0,
               'axes': 'TYX',
                 'Labels': fname_list,
                     })
        
        print("Done.")
        break
        
window.close()

### 考察  
* 今までFIJIのマクロで行っていた処理をほぼ完全にPythonで再現することに成功した。  
* 解析対象の細胞の大きさはマクロと同じく400-3000にしているが、解析結果（fociを持つ細胞の割合）はFIJIマクロとPythonで若干異なる。  
これは細胞を検出するための手順の違いによるものだと考えられる。
* Pythonの方がfociを持つ細胞の割合が10%程度高く見積もられる。
* また細胞のラベルも大きくなる傾向があるので、300-2000くらいが大きさの範囲として適切かもしれない。
* Pythonでは35個のファイルを処理するのに1分程度しかかからない。同様の処理でFIJIでは20−30分かかっていたので、少なくとも20倍の高速化を実現。